In [1]:
import config

soynlp=0.0.49
added lovit_textmining_dataset


## String distance

Levenshtein distance 는 한 string s1 이 다른 string s2 로 바뀌기 위한 수정 (edit) 횟수를 두 글자의 거리로 정의합니다. 그렇기 때문에 edit distance 라고도 불립니다. Editing 의 종류에는 새로운 글자를 삽입하는 insert, 한 글자를 제거하는 delete, 한 글자를 다른 글자로 변환되는 substitution 세 가지로 나뉩니다.

Edit distance 는 editing 의 unit 을 글자, 초/중/종성, 단어로 정의할 수 있습니다. 이 튜토리얼에서는 editing 단위에 따른 edit distance 에 대하여 알아봅니다.

1. Edit distance (Levenshtein distance)
1. Token edit distance
1. Jamo edit distance
1. Token - Jamo edit distance

그 외에도 Cosine distance 와 Jaccard distance 를 직접 구현해 봅니다.

## Levenshtein distance

기본적인 Levenshtein distance (Edit distance)를 먼저 만든 뒤, 하나씩 변형을 해보겠습니다. 

코드는 [이 주소][lev]를 참고하여 가져왔습니다. 두 str s1, s2에 대하여 s1의 길이가 s2보다 길거나 같다고 가정합니다. 그래서 len(s1) < len(s2)를 확인하여, s2의 길이가 더 길 경우에는 반대로 입력합니다. s1에서 s2로 바뀌거나, s2에서 s1으로 바뀌는 비용은 같기 때문입니다. 

[lev]: https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python

Python에서는 Boolean 비교의 결과가 0과 1로 출력됩니다. True는 1이며, 0은 False입니다. levenshtein 구현의 Line number 15에서 c1과 c2가 다르면 1의 substitution cost를 주는 것을 볼 수 있습니다. 

```python
substitutions = previous_row[j] + (c1 != c2)
```

In [2]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

s1 과 s2 는 str 형식이어도 되며, list of str 이어도 됩니다. s1, s2 가 str 형식이라면 글자 단위의 edit distance 입니다

In [3]:
test_pairs = [
    ('머신러닝 텍스트 마이닝', '머신러닝 택스트 마이니'), # (텍 <-> 택, 닝 <-> 니)
    ('머신러닝 텍스트 마이닝', '머신러닝 마이닝 텍스트'), # (텍스트 <-> 마이닝)
    ('머신러닝 텍스트 마이닝', '머신텍스트 마이닝') # (러닝_ delete)
]

for pair in test_pairs:
    print('pair={}, distance = {}'.format(pair, levenshtein(pair[0], pair[1])))

pair=('머신러닝 텍스트 마이닝', '머신러닝 택스트 마이니'), distance = 2
pair=('머신러닝 텍스트 마이닝', '머신러닝 마이닝 텍스트'), distance = 6
pair=('머신러닝 텍스트 마이닝', '머신텍스트 마이닝'), distance = 3


list of str 이라면 단어 단위의 edit distance 가 됩니다.

In [4]:
for pair in test_pairs:
    distance = levenshtein(pair[0].split(),pair[1].split())
    print('pair={}, distance = {}'.format(pair, distance))

pair=('머신러닝 텍스트 마이닝', '머신러닝 택스트 마이니'), distance = 2
pair=('머신러닝 텍스트 마이닝', '머신러닝 마이닝 텍스트'), distance = 2
pair=('머신러닝 텍스트 마이닝', '머신텍스트 마이닝'), distance = 2


사용자가 substitutions 의 비용을 설정할 수도 있습니다. c1 에서 c2 로 바뀌는 비용을 cost 에 넣어둡니다. get_cost 함수를 이용하여 c1 이 c2 로 바뀔 때의 비용을 가져옵니다. 만약 설정된 값이 없으면 1 을 부여합니다.

```python
def levenshtein(s1, s2, cost=None):
    # ...
    def get_cost(c1, c2, cost):
        return 0 if (c1 == c2) else cost.get((c1, c2), 1)

    for i, c1 in enumerate(s1):
        for j, c2 in enumerate(s2):
            substitutions = previous_row[j] + get_cost(c1, c2, cost)
```

In [5]:
def levenshtein(s1, s2, cost=None):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)
    
    if cost == None:
        cost = {}
    
    def get_cost(c1, c2, cost):
        return 0 if (c1 == c2) else cost.get((c1, c2), 1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + get_cost(c1, c2, cost)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [6]:
dist = levenshtein('텍스트 마이닝', '택스트 마이닝')
print('w/o cost\t텍스트 마이닝 --> 택스트 마이닝 : {}'.format(dist))

dist = levenshtein('텍스트 마이닝', '택스트 마이닝', {('텍', '택'):0.1})
print('with cost\t텍스트 마이닝 --> 택스트 마이닝 : {}'.format(dist))

w/o cost	텍스트 마이닝 --> 택스트 마이닝 : 1
with cost	텍스트 마이닝 --> 택스트 마이닝 : 0.1


FastText 의 실습에서 이용하였던 decompose 함수를 이용하여 한글을 초/중/종성으로 분리할 수 있습니다.

In [7]:
from soynlp.hangle import character_is_complete_korean as is_hangle
from soynlp.hangle import decompose


print(is_hangle('ㄱ'))
print(is_hangle('a'))
print(is_hangle('감'))
print(decompose('감'))

False
False
True
('ㄱ', 'ㅏ', 'ㅁ')


글자의 substitution 비용에서 한글을 초/중/종성으로 분해하는 기능을 넣으면 jamo levenshtein distance 를 정의할 수 있습니다.

get_cost 함수만 다시 정의하면 됩니다. c1 과 c2 중 하나 이상이 한글이 아니면 1 을 return 하고, 둘 모두가 한글이라면 초/중/종성을 분리하여 leveshtein distance 를 계산합니다. 1 음절의 한글은 3 개의 글자 (초/중/종)를 가지기 때문에 이 거리를 3 으로 나누어 return 합니다.

```python
def jamo_levenshtein(s1, s2):
    # ...
    def get_cost(c1, c2):
        if not is_hangle(c1) or not is_hangle(c2):
            return 1
        return 0 if (c1 == c2) else levenshtein(decompose(c1), decompose(c2))/3
```

In [8]:
def jamo_levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)
    
    def get_cost(c1, c2):
        if c1 == c2:
            return 0
        if not is_hangle(c1) or not is_hangle(c2):
            return 1
        return levenshtein(decompose(c1), decompose(c2))/3

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + get_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

jamo_levenshtein('텍스트 마이닝', '택스트 마이닝')

0.3333333333333333

## Jaccard distance

Jaccard distance 는 unit 들에 대하여 set 의 1 - (intersection / union) 값입니다. 

주어진 단어 s1, s2에 대하여 unit을 만드는 함수를 lambda를 통하여 정의하였습니다. 

```python
s1_set = unify(s1)
```
 
이 부분을 통하여 주어진 s1, s2 는 unit set 으로 바뀌게 되며, str 을 set 처리하면 글자의 set 이 출력됩니다. set 은 str 의 글자들의 unique set 을 만듭니다.

In [9]:
print(set('abcc'))
print(set('a'))

{'a', 'c', 'b'}
{'a'}


In [10]:
def jaccard_distance(s1, s2, unitfy=lambda x:set(x), debug=False):
    if (not s1) or (not s2):
        return 1
    
    s1_set = unitfy(s1)
    s2_set = unitfy(s2)

    if debug:
        print('  - unit of s1 = {}'.format(s1_set))
        print('  - unit of s2 = {}'.format(s2_set))

    intersection = s1_set.intersection(s2_set)
    union = s1_set.union(s2_set)
    return 1 - len(intersection) / len(union)

In [11]:
s1 = '자살금지'
s2 = '지금살자'

print('unigram unit')
dist = jaccard_distance(s1, s2, debug=True)
print('distance = {}'.format(dist), end='\n\n')


bigram = lambda x:set([x[i:i+2] for i in range(len(x)-1)])
print('bigram unit')
dist = jaccard_distance(s1, s2, bigram, debug=True) 
print('distance = {}'.format(dist), end='\n\n')

dist = levenshtein(s1, s2)
print('edit distance = {}'.format(dist))

unigram unit
  - unit of s1 = {'살', '금', '지', '자'}
  - unit of s2 = {'지', '자', '살', '금'}
distance = 0.0

bigram unit
  - unit of s1 = {'자살', '살금', '금지'}
  - unit of s2 = {'지금', '금살', '살자'}
distance = 1.0

edit distance = 4


## Cosine

Jaccard 는 각 unit 이 몇 번 등장했는지에 대한 정보를 이용하지 않습니다. Boolean vector 와 같습니다. Cosine distance 는 한 unit 의 등장 횟수도 이용합니다. unitfy 함수를 Counter 를 이용할 수 있습니다.

In [12]:
from collections import Counter

Counter('abcc')

Counter({'a': 1, 'b': 1, 'c': 2})

cosine 은 두 벡터의 내적을 각 벡터의 L2 norm 으로 나눠준 값입니다. d1 과 d2 의 빈도수 제곱의 합의 1/2 승이 L2 norm 입니다.

In [13]:
import numpy as np

def cosine_distance(s1, s2, unitfy=lambda x:Counter(x), debug=False):
    """distance = 1 - cosine similarity; [0, 2]
    """
    
    if (not s1) or (not s2):
        return 2
    
    d1 = unitfy(s1)
    d2 = unitfy(s2)

    if debug:
        print('  - unit of s1 = {}'.format(d1))
        print('  - unit of s2 = {}'.format(d2))

    prod = 0
    for c1, f in d1.items():
        prod += (f * d2.get(c1, 0))

    norm1 = sum(v**2 for v in d1.values()) ** (1/2)
    norm2 = sum(v**2 for v in d2.values()) ** (1/2)

    cos_sim = prod / (norm1 * norm2)
    cos_dist = 1 - cos_sim
    return cos_dist

In [14]:
s1 = '데이터마이닝'
s2 = '대이타마이닝'


dist = cosine_distance(s1, s2, debug=True)
print('distance = {:.3}'.format(dist), end='\n\n')

dist = cosine_distance(s1, s2, lambda x:Counter(bigram(x)), debug=True)
print('distance = {:.3}'.format(dist), end='\n\n')

print('jaccard distance = {}'.format(jaccard_distance(s1, s2)))

  - unit of s1 = Counter({'이': 2, '데': 1, '터': 1, '마': 1, '닝': 1})
  - unit of s2 = Counter({'이': 2, '대': 1, '타': 1, '마': 1, '닝': 1})
distance = 0.25

  - unit of s1 = Counter({'마이': 1, '터마': 1, '이터': 1, '이닝': 1, '데이': 1})
  - unit of s2 = Counter({'마이': 1, '대이': 1, '타마': 1, '이닝': 1, '이타': 1})
distance = 0.6

jaccard distance = 0.5714285714285714


## soynlp

위에서 실습한 기능들은 모두 soynlp에 구현해 두었습니다. 함수를 튜닝하실 때에는 코드를 직접 만드시고, 있는 기능을 쓰실 때에는 import 해서 쓰셔도 됩니다. 

In [15]:
from soynlp.hangle import levenshtein, jamo_levenshtein, cosine_distance, jaccard_distance

In [16]:
s1 = '데이터마이닝'
s2 = '대이타마이닝'

print('levenshtein:         %.3f' % levenshtein(s1, s2))
print('jamo levenshtein:    %.3f' % jamo_levenshtein(s1, s2))

levenshtein:         2.000
jamo levenshtein:    0.667


In [17]:
print('cosine w 1syllable:  %.3f' % cosine_distance(s1, s2))
print('cosine w 2syllable:  %.3f' %  cosine_distance(s1, s2, lambda x:Counter(bigram(x))))

cosine w 1syllable:  0.250
cosine w 2syllable:  0.600


In [18]:
print('jaccard w 1syllable: %.3f' %  jaccard_distance(s1, s2))
print('jaccard w 2syllable: %.3f' %  jaccard_distance(s1, s2, bigram))

jaccard w 1syllable: 0.571
jaccard w 2syllable: 0.750
